In [1]:
%autosave 60

Autosaving every 60 seconds


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import requests
from io import StringIO
from bs4 import BeautifulSoup


In [4]:
#sp-500 historical returns
url = "https://www.macrotrends.net/2526/sp-500-historical-annual-returns"
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
}

#get data
response = requests.get(url, headers=headers)
tables = pd.read_html(response.text)

df=tables[0]

#flatten df to single-index
df.columns = ['_'.join(col).strip() for col in df.columns] # Resulting single-layer DataFrame print(multi_col_df)

#rename columns
for column in df.columns:
    df = df.rename(columns={column: column.replace('S&P 500 Index - Historical Annual Data_', '')})

#reset index so first date is first index
df = df.sort_values(df.columns[0], ascending=True)
df = df.reset_index()
df = df.drop(columns=['index'])

display(df)

<ipython-input-4-466e80b0fa81>:9: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  tables = pd.read_html(response.text)


,Year,Average Closing Price,Year Open,Year High,Year Low,Year Close,Annual % Change
0,1928,19.94,17.76,24.35,16.95,24.35,37.88%
1,1929,26.19,24.81,31.86,17.66,21.45,-11.91%
2,1930,21.00,21.18,25.92,14.44,15.34,-28.48%
3,1931,13.66,15.85,18.17,7.72,8.12,-47.07%
4,1932,6.92,7.82,9.31,4.40,6.89,-15.15%
...,...,...,...,...,...,...,...
92,2020,3217.86,3257.85,3756.07,2237.40,3756.07,16.26%
93,2021,4273.41,3700.65,4793.06,3700.65,4766.18,26.89%
94,2022,4097.49,4796.56,4796.56,3577.03,3839.50,-19.44%
95,2023,4283.73,3824.14,4783.35,3808.10,4769.83,24.23%


In [5]:
# 10-year treasury yield

url2 = 'https://fred.stlouisfed.org/graph/fredgraph.csv?id=RIFLGFCY10NA'

# get data
response2 = requests.get(url2)
treasury_data = StringIO(response2.text)

# read data into dataframe
treasury_df = pd.read_csv(treasury_data)

# date into column
treasury_df['Year'] = pd.to_datetime(treasury_df['DATE'])

# convert to year
treasury_df['Year'] = treasury_df['Year'].dt.year

# rename treasury yield column
treasury_df.rename(columns={'RIFLGFCY10NA': 'yield_10yr'}, inplace=True)

# ascending year (to match the previous data)
treasury_df = treasury_df.sort_values('Year', ascending=True)

# merge dataframes on 'Year'
df = pd.merge(df, treasury_df[['Year', 'yield_10yr']], how='left')

df


,Year,Average Closing Price,Year Open,Year High,Year Low,Year Close,Annual % Change,yield_10yr
0,1928,19.94,17.76,24.35,16.95,24.35,37.88%,NaN
1,1929,26.19,24.81,31.86,17.66,21.45,-11.91%,NaN
2,1930,21.00,21.18,25.92,14.44,15.34,-28.48%,NaN
3,1931,13.66,15.85,18.17,7.72,8.12,-47.07%,NaN
4,1932,6.92,7.82,9.31,4.40,6.89,-15.15%,NaN
...,...,...,...,...,...,...,...,...
92,2020,3217.86,3257.85,3756.07,2237.40,3756.07,16.26%,0.89
93,2021,4273.41,3700.65,4793.06,3700.65,4766.18,26.89%,1.45
94,2022,4097.49,4796.56,4796.56,3577.03,3839.50,-19.44%,2.95
95,2023,4283.73,3824.14,4783.35,3808.10,4769.83,24.23%,3.96


In [6]:
#VIX - annualized

#this code fetches daily VIX index
#url3 = 'https://fred.stlouisfed.org/graph/fredgraph.csv?id=VIXCLS'
#response3 = requests.get(url3)
#vix_data = StringIO(response3.text)

# Read the CSV data into a pandas DataFrame
vix_df = pd.read_csv('data/VIX.csv')

# Process data
vix_df['Year'] = pd.to_datetime(vix_df['DATE'])
vix_df['Year'] = vix_df['Year'].dt.year
vix_df = vix_df.sort_values('Year', ascending=True)
vix_df.rename(columns={'VIXCLS': 'vix_annualized'}, inplace=True)

# Add to dataframe
df = pd.merge(df, vix_df[['Year', 'vix_annualized']], how='left')

# Display the first few rows to confirm early data
print("Earliest data points:")
print(vix_df.head())

Earliest data points:
         DATE  vix_annualized  Year
0  1990-01-01       23.063478  1990
1  1991-01-01       18.373373  1991
2  1992-01-01       15.452047  1992
3  1993-01-01       12.686245  1993
4  1994-01-01       13.925516  1994


In [7]:
# % change in GDP Annual
url4 = 'https://fred.stlouisfed.org/graph/fredgraph.csv?id=A191RP1A027NBEA'

#get data
response4 = requests.get(url4)
gdp_data = StringIO(response4.text)
gdp_df = pd.read_csv(gdp_data)

#process data
gdp_df['Year'] = pd.to_datetime(gdp_df['DATE'])
gdp_df['Year'] = gdp_df['Year'].dt.year
gdp_df = gdp_df.sort_values('Year', ascending=True)
gdp_df.rename(columns={'A191RP1A027NBEA': 'gdp_growth'}, inplace=True)

#add to dataframe
df = pd.merge(df, gdp_df[['Year', 'gdp_growth']], how='left')

df

,Year,Average Closing Price,Year Open,Year High,Year Low,Year Close,Annual % Change,yield_10yr,vix_annualized,gdp_growth
0,1928,19.94,17.76,24.35,16.95,24.35,37.88%,NaN,NaN,NaN
1,1929,26.19,24.81,31.86,17.66,21.45,-11.91%,NaN,NaN,NaN
2,1930,21.00,21.18,25.92,14.44,15.34,-28.48%,NaN,NaN,-11.9
3,1931,13.66,15.85,18.17,7.72,8.12,-47.07%,NaN,NaN,-16.0
4,1932,6.92,7.82,9.31,4.40,6.89,-15.15%,NaN,NaN,-23.1
...,...,...,...,...,...,...,...,...,...,...
92,2020,3217.86,3257.85,3756.07,2237.40,3756.07,16.26%,0.89,29.251304,-0.9
93,2021,4273.41,3700.65,4793.06,3700.65,4766.18,26.89%,1.45,19.656151,10.9
94,2022,4097.49,4796.56,4796.56,3577.03,3839.50,-19.44%,2.95,25.637148,9.8
95,2023,4283.73,3824.14,4783.35,3808.10,4769.83,24.23%,3.96,16.849027,6.6


In [ ]:
# Unemployment Rate - annualized

# fetches monthly unemployment rate
#url5 = 'https://fred.stlouisfed.org/graph/fredgraph.csv?id=UNRATE'
#response5 = requests.get(url5)
#unemployment_data = StringIO(response5.text)

unemployment_df = pd.read_csv('data/unemployment_rate.csv')

# function to process Fed data
def process_fed_data(df, main_column_name):
    # rename and convert date column to year
    df['Year'] = pd.to_datetime(df['DATE'])
    df['Year'] = df['Year'].dt.year
    #sort values
    df.sort_values('Year', ascending=True)
    # rename main column to something normal
    df.columns.values[1] = main_column_name

process_fed_data(unemployment_df, 'unemployment')

print(unemployment_df.head())

In [ ]:
# Consumer Price Index - annual

url6 = 'https://fred.stlouisfed.org/graph/fredgraph.csv?id=FPCPITOTLZGUSA'
response6 = requests.get(url6)
cpi_data = StringIO(response6.text)
cpi_df = pd.read_csv(cpi_data)

cpi_df['DATE'] = pd.to_datetime(cpi_df['DATE'])
cpi_df = cpi_df.sort_values('DATE', ascending=True)

print(cpi_df.head())

In [ ]:
# Fed Funds Rate- daily

url7 = 'https://fred.stlouisfed.org/graph/fredgraph.csv?id=FEDFUNDS'

response7 = requests.get(url7)
fedfunds_data = StringIO(response7.text)
fedfunds_df = pd.read_csv(fedfunds_data)

fedfunds_df['DATE'] = pd.to_datetime(fedfunds_df['DATE'])
fedfunds_df = fedfunds_df.sort_values('DATE', ascending=True)

print(fedfunds_df.head())

In [ ]:
# Federal Surplus or Deficit - annual fiscal year
url8 = 'https://fred.stlouisfed.org/graph/fredgraph.csv?id=FYFSD'

response8 = requests.get(url8)
deficit_data = StringIO(response8.text)
deficit_df = pd.read_csv(deficit_data)

deficit_df['DATE'] = pd.to_datetime(deficit_df['DATE'])
deficit_df = deficit_df.sort_values('DATE', ascending=True)

print(deficit_df.head())

In [ ]:
# *Earnings Report Sentiment (couldn't find it online and on report)@adam

In [ ]:
# *Average EPS--wasn't sure how to get this either. Something we have to do manually? @adam

In [ ]:
# U.S. Crime Rate- annual


crime_data = [
    [2021, 6.81, 6.02],
    [2020, 6.42, 28.78],
    [2019, 4.99, 1.14],
    [2018, 4.93, -5.99],
    [2017, 5.24, -1.46],
    [2016, 5.32, 8.76],
    [2015, 4.89, 11.25],
    [2014, 4.40, -1.90],
    [2013, 4.48, -4.44],
    [2012, 4.69, 0.44],
    [2011, 4.67, -1.27],
    [2010, 4.73, -5.22],
    [2009, 4.99, -7.33],
    [2008, 5.39, -4.80],
    [2007, 5.66, -2.02],
    [2006, 5.77, 2.60],
    [2002, 5.63, -15.58],
    [2001, 6.67, 20.80],
    [2000, 5.52, -0.73],
    [1999, 5.56, -9.65],
    [1998, 6.15, -7.94],
    [1997, 6.68, -8.48],
    [1996, 7.30, -10.20],
    [1995, 8.13, -8.56],
    [1994, 8.89, -6.16],
    [1993, 9.48, 1.79],
    [1992, 9.31, -5.18],
    [1991, 9.82, 3.94],
    [1990, 9.45, 3.94]]

crime_df = pd.DataFrame(crime_data, columns=['DATE', 'CRIME_RATE', 'ANNUAL_CHANGE'])
crime_df['DATE'] = pd.to_datetime(crime_df['DATE'], format='%Y')
crime_df = crime_df.sort_values('DATE', ascending=True)

print(crime_df.head())



In [ ]:
#Presidental List--have to physically enter it? @adam, what I did is prob not ideal lol, manually enter it, with party as binary? LOL
#@adam @Samaga

In [ ]:
url13 = 'https://www.macrotrends.net/global-metrics/countries/USA/united-states/military-spending-defense-budget'

#no clue why the similar code wasn't working. I also tried the web scrapping method from first pset and that wasn't working either.

In [ ]:
# U.S. Birth Rate - annual
url14 = 'https://www.macrotrends.net/global-metrics/countries/usa/united-states/birth-rate'

response14 = requests.get(url14)
tables = pd.read_html(response14.text)

# The historical data is likely in the last table
birth_rate_df = tables[-1]

birth_rate_df.columns = ['Year', 'Birth_Rate', 'Growth_Rate']
birth_rate_df['Year'] = pd.to_datetime(birth_rate_df['Year'], format='%Y')
birth_rate_df = birth_rate_df.sort_values('Year', ascending=True)

print(birth_rate_df.head())



In [ ]:
# prompt: write this file into a dataframe sp-500-historical-annual-returns.csv


# yield_10yr = pd.read_csv('data/10yr_treasury_yield.csv')

# real_gdp_growth = pd.read_csv('data/real_gdp_growth.csv')

# inflation = pd.read_csv('data/inflation.csv')

# deficit = pd.read_csv('data/deficit.csv')

# pop_growth = pd.read_csv('data/pop_growth.csv')

# interest_rate = pd.read_csv('data/interest_rate.csv')
